<a id='da'></a>
# tabular data aggregation|

[libraries](#da-libraries)   
[variables](#da-variables)   
[categoricals](#da-categoricals)   
[directories](#da-directories)   
[read](#da-read)   
[transform](#da-transform)   
[statistics](#da-statistics)  
[agg](#da-agg)  
[groupby](#da-groupby)  
[groupby.agg](#da-groupby.agg)  
[resample](#da-resample)  
[export](#da-export)  

- Aggregation methods

- sum, mean, median, min, max, count, std, var, describe, quantile
- The technical definition of an aggregation is when a sequence of values is summarized by a single number.

- axis parameter
  - axis 0
    Default axis for most DataFrame methods
    Also referenced by the string 'index'
    Operations happen vertically, up and down the columns
    Example - df.sum() computes the sum of each column individually
  - axis 1
    Also referenced by the string 'columns'
    Operations happen horizontally, left to right across each row
    Example - df.sum(axis='columns') computes the sum of each row individually
    
- numeric_only=True
  - eliminates time-consuming calculation of string aggregations (often dropped as nuisance columns anyways)

- Aggregation Methods (minimally sufficient pandas)
  - describe
  - count, sum, max, min
  - idxmax, idxmin
  - all, any
  - mean, median, mode, std, var
  - nunique

## project

In [1]:
project_id = 'gta'

<a id='da-libraries'></a>
## libraries

In [2]:
import os
import glob
import calendar
import pandas as pd

In [3]:
import warnings 
from functools import partial 
from IPython.core.interactiveshell import InteractiveShell 

### parameters

In [4]:
# ignore warnings 
warnings.filterwarnings('ignore')

In [5]:
# shorter traceback error messages
get_ipython().showtraceback = partial(get_ipython().showtraceback,exception_only=True)

In [6]:
# shows result of cell without needing print
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

In [7]:
from pandas.api.types import CategoricalDtype

<a id='da-variables'></a>
## variables

[Return to Start of Notebook](#da)  

In [8]:
months = [month for month in calendar.month_abbr if month]

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [9]:
seasons = ['Spring', 'Summer', 'Autumn','Winter']

['Spring', 'Summer', 'Autumn', 'Winter']

<a id='da-categoricals'></a>
## categoricals

[Return to Start of Notebook](#da)  

In [10]:
months_cat = CategoricalDtype(months, ordered=True)

CategoricalDtype(categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
                  'Sep', 'Oct', 'Nov', 'Dec'],
, ordered=True)

In [11]:
seasons_cat = CategoricalDtype(seasons, ordered=True)

CategoricalDtype(categories=['Spring', 'Summer', 'Autumn', 'Winter'], ordered=True)

<a id='da-directories'></a>
## directories

[Return to Start of Notebook](#da)  

In [12]:
home_dir = os.path.expanduser("~")
project_dir = os.path.join(home_dir, project_id)
data_dir = os.path.join(project_dir, 'data')
restructured_data_dir = os.path.join(data_dir, 'restructured-data')
aggregated_data_dir = os.path.join(data_dir, 'aggregated-data') 

'/Users/rkforest/gta/data/aggregated-data'

In [13]:
def create_directory(directory_name):
    if not os.path.exists(directory_name):
        print('Created new directory: ', directory_name)
        os.mkdir(directory_name)

In [14]:
create_directory(aggregated_data_dir)

Created new directory:  /Users/rkforest/gta/data/aggregated-data


In [15]:
os.chdir(project_dir)
os.getcwd()

'/Users/rkforest/gta'

### paths

In [16]:
file_paths =  sorted(glob.glob(os.path.join(restructured_data_dir, '*.csv')))
[print(path) for path in file_paths];

/Users/rkforest/gta/data/restructured-data/global_data_by_month.csv
/Users/rkforest/gta/data/restructured-data/global_data_by_year.csv
/Users/rkforest/gta/data/restructured-data/hemisphere_data_by_month.csv
/Users/rkforest/gta/data/restructured-data/nh_data_by_month.csv
/Users/rkforest/gta/data/restructured-data/sh_data_by_month.csv
/Users/rkforest/gta/data/restructured-data/zonal_data_by_year.csv


<a id='da-read'></a>
## read

[Return to Start of Notebook](#da)  

In [17]:
df_global_by_month = pd.read_csv(file_paths[0],
                                 parse_dates=['date'],  
                                 index_col='date') 
df_global_by_month.head(1)

,year,month,season,area,anomaly
date,,,,,
1880-01-01,1880,Jan,Winter,GLB,-0.18


In [18]:
df_global_by_year = pd.read_csv(file_paths[1],
                                parse_dates=['year'],  
                                index_col='year') 
df_global_by_year.head(1)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
year,,,,,,,,,,,,
1880-01-01,-0.18,-0.24,-0.09,-0.16,-0.1,-0.21,-0.18,-0.1,-0.15,-0.23,-0.22,-0.17


In [19]:
df_hemisphere_by_year = pd.read_csv(file_paths[2],
                                    parse_dates=['date'],  
                                    index_col='date') 
df_hemisphere_by_year.head(1)

,season,month,area,anomaly
date,,,,
1880-01-01,Winter,Jan,NH,-0.36


<a id='da-transform'></a>
## transform

[Return to Start of Notebook](#da)  

In [20]:
df_global_by_month['month'] = df_global_by_month['month'].astype(months_cat)
df_global_by_month['season'] = df_global_by_month['season'].astype(seasons_cat)
df_global_by_month.dtypes

year          int64
month      category
season     category
area         object
anomaly     float64
dtype: object

<a id='da-statistics'></a>
## statistics

[Return to Start of Notebook](#da)  

- sum, mean, median, min, max, count, std, var, describe, quantile

### single column

In [21]:
df_global_by_month['anomaly'].mean()

0.062390924956369984

### all df columns

In [22]:
months_mean = df_global_by_year.mean()

Jan    0.064236
Feb    0.071806
Mar    0.090069
Apr    0.056783
May    0.047343
Jun    0.033077
Jul    0.056573
Aug    0.055035
Sep    0.058322
Oct    0.084755
Nov    0.077762
Dec    0.052657
dtype: float64

In [23]:
type(months_mean)

pandas.core.series.Series

#### numeric only

In [24]:
df_global_by_month.mean(numeric_only=True)

year       1951.125654
anomaly       0.062391
dtype: float64

<a id='da-agg'></a>
## .agg
- specify columns
- specify statistics
- specify axis

[Return to Start of Notebook](#da)  

In [25]:
column_stats = df_global_by_month\
                    [['anomaly']]\
                    .agg(['sum','mean','max','min','median','count'],
                    axis=0)

,anomaly
sum,107.250000
mean,0.062391
max,1.370000
min,-0.820000
median,-0.030000
count,1719.000000


In [26]:
column_stats = df_global_by_month\
                    [['anomaly']]\
                    .agg(['sum','mean','max','min','median','count'],
                    axis=0).T

,sum,mean,max,min,median,count
anomaly,107.25,0.062391,1.37,-0.82,-0.03,1719.0


In [27]:
all_column_stats = df_global_by_year\
                    .agg(['mean'],
                    axis=0)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
mean,0.064236,0.071806,0.090069,0.056783,0.047343,0.033077,0.056573,0.055035,0.058322,0.084755,0.077762,0.052657


In [28]:
row_stats = df_global_by_month\
                    [['anomaly']]\
                    .agg(['sum','mean','max','min','median','count'],
                    axis=1)
row_stats.head(3)

,sum,mean,max,min,median,count
date,,,,,,
1880-01-01,-0.18,-0.18,-0.18,-0.18,-0.18,1.0
1880-02-01,-0.24,-0.24,-0.24,-0.24,-0.24,1.0
1880-03-01,-0.09,-0.09,-0.09,-0.09,-0.09,1.0


<a id='da-groupby'></a>
## groupby

[Return to Start of Notebook](#da)  

### group using .groupby()

In [29]:
grouped_by_season = df_global_by_month.groupby('season')
type(grouped_by_season)

pandas.core.groupby.generic.DataFrameGroupBy

In [30]:
grouped_by_month = df_global_by_month.groupby('month')
type(grouped_by_month)

pandas.core.groupby.generic.DataFrameGroupBy

### group keys

In [31]:
grouped_by_season.groups.keys()

dict_keys(['Spring', 'Summer', 'Autumn', 'Winter'])

### group len

In [32]:
for idx, grp in grouped_by_season:
    print(idx,len(grp))

Spring 430
Summer 429
Autumn 429
Winter 431


### get group

In [33]:
key = "Spring"
group1 = grouped_by_season.get_group(key)
group1.head(3)

,year,month,season,area,anomaly
date,,,,,
1880-03-01,1880,Mar,Spring,GLB,-0.09
1880-04-01,1880,Apr,Spring,GLB,-0.16
1880-05-01,1880,May,Spring,GLB,-0.10


In [34]:
type(group1)

pandas.core.frame.DataFrame

In [35]:
for idx, grp in grouped_by_month:
    print(idx,grp['anomaly'].mean())

Jan 0.0642361111111111
Feb 0.07180555555555557
Mar 0.09006944444444447
Apr 0.056783216783216815
May 0.047342657342657336
Jun 0.03307692307692311
Jul 0.05657342657342659
Aug 0.05503496503496503
Sep 0.05832167832167831
Oct 0.08475524475524476
Nov 0.07776223776223774
Dec 0.05265734265734266


### apply aggregation to groups
(results combined into new df)

In [36]:
season_avg = grouped_by_season[['anomaly']].mean()

,anomaly
season,
Spring,0.064791
Summer,0.048228
Autumn,0.073613
Winter,0.062923


In [37]:
print(type(season_avg))

<class 'pandas.core.frame.DataFrame'>


### chain as single step

In [38]:
season_avg = df_global_by_month.groupby('season')[['anomaly']].mean()
season_avg

,anomaly
season,
Spring,0.064791
Summer,0.048228
Autumn,0.073613
Winter,0.062923


In [39]:
# max for each month
months_sorted = df_global_by_month[['anomaly','month']] \
    .sort_values('anomaly',ascending=False) \
    .drop_duplicates(subset=['month'])
months_sorted.head(3)

,anomaly,month
date,,
2016-02-01,1.37,Feb
2016-03-01,1.36,Mar
2016-01-01,1.17,Jan


In [40]:
# min for each season
months_sorted = df_global_by_month[['anomaly','season']] \
    .sort_values('anomaly',ascending=True) \
    .drop_duplicates(subset=['season'])

,anomaly,season
date,,
1916-12-01,-0.82,Winter
1917-03-01,-0.63,Spring
1912-09-01,-0.58,Autumn
1912-08-01,-0.54,Summer


<a id='da-groupby.agg'></a>
## .groupby().agg()

[Return to Start of Notebook](#da)  

#### group by 
 - specify groupby column
 - sepcify data column
 - specify statistics
 - specify new column names

In [41]:
df_global_grouped_by_month = df_global_by_month\
    .groupby('month')\
    .agg(
        avg_anomaly=('anomaly', 'mean'),
        max_anomaly=('anomaly', 'max'),
        min_anomaly=('anomaly', 'min'),
        cnt_anomaly=('anomaly', 'count'))
df_global_grouped_by_month

,avg_anomaly,max_anomaly,min_anomaly,cnt_anomaly
month,,,,
Jan,0.064236,1.17,-0.81,144
Feb,0.071806,1.37,-0.63,144
Mar,0.090069,1.36,-0.63,144
Apr,0.056783,1.13,-0.59,143
May,0.047343,1.02,-0.55,143
Jun,0.033077,0.92,-0.52,143
Jul,0.056573,0.94,-0.51,143
Aug,0.055035,1.02,-0.54,143
Sep,0.058322,0.99,-0.58,143


In [42]:
df_global_grouped_by_season = df_global_by_month\
    .groupby('season')\
    .agg(
        avg_anomaly=('anomaly', 'mean'),
        max_anomaly=('anomaly', 'max'),
        min_anomaly=('anomaly', 'min'),
        cnt_anomaly=('anomaly', 'count'))
df_global_grouped_by_season

,avg_anomaly,max_anomaly,min_anomaly,cnt_anomaly
season,,,,
Spring,0.064791,1.36,-0.63,430
Summer,0.048228,1.02,-0.54,429
Autumn,0.073613,1.11,-0.58,429
Winter,0.062923,1.37,-0.82,431


#### dictionary syntax

In [43]:
grouped_by_season = df_global_by_month\
    .groupby('season')\
    .agg(
        {'anomaly': ['mean', 'max', 'min', 'count']})
grouped_by_season

anomaly                  
            mean   max   min count
season                            
Spring  0.064791  1.36 -0.63   430
Summer  0.048228  1.02 -0.54   429
Autumn  0.073613  1.11 -0.58   429
Winter  0.062923  1.37 -0.82   431

##### .droplevel()

In [44]:
grouped_by_season.columns = grouped_by_season.columns.droplevel(0)
grouped_by_season

,mean,max,min,count
season,,,,
Spring,0.064791,1.36,-0.63,430
Summer,0.048228,1.02,-0.54,429
Autumn,0.073613,1.11,-0.58,429
Winter,0.062923,1.37,-0.82,431


#### brackets syntax

In [45]:
grouped_by_season = df_global_by_month\
    .groupby('season')['anomaly']\
    .agg(
        ['mean', 'max', 'min', 'count'])
grouped_by_season

,mean,max,min,count
season,,,,
Spring,0.064791,1.36,-0.63,430
Summer,0.048228,1.02,-0.54,429
Autumn,0.073613,1.11,-0.58,429
Winter,0.062923,1.37,-0.82,431


#### do not use group column as index

In [46]:
grouped_by_season = df_global_by_month\
    .groupby('season', as_index=False)['anomaly']\
    .agg(
        ['mean', 'max', 'min', 'count'])
grouped_by_season

,season,mean,max,min,count
0,Spring,0.064791,1.36,-0.63,430
1,Summer,0.048228,1.02,-0.54,429
2,Autumn,0.073613,1.11,-0.58,429
3,Winter,0.062923,1.37,-0.82,431


#### describe

In [47]:
grouped_by_season = df_global_by_month\
    .groupby('season')['anomaly']\
    .describe()
grouped_by_season

,count,mean,std,min,25%,50%,75%,max
season,,,,,,,,
Spring,430.0,0.064791,0.398307,-0.63,-0.23,-0.03,0.29,1.36
Summer,429.0,0.048228,0.357961,-0.54,-0.22,-0.04,0.25,1.02
Autumn,429.0,0.073613,0.366876,-0.58,-0.19,-0.02,0.23,1.11
Winter,431.0,0.062923,0.413552,-0.82,-0.23,-0.03,0.32,1.37


#### group with multiple columns
observed = True

In [48]:
grouped_by_season = df_global_by_month\
    .groupby(['season','month'],observed=True)\
    .agg(
        {'anomaly': ['mean', 'max', 'min', 'count']})
grouped_by_season.columns = grouped_by_season.columns.droplevel(0)
grouped_by_season

mean   max   min  count
season month                             
Spring Mar    0.090069  1.36 -0.63    144
       Apr    0.056783  1.13 -0.59    143
       May    0.047343  1.02 -0.55    143
Summer Jun    0.033077  0.92 -0.52    143
       Jul    0.056573  0.94 -0.51    143
       Aug    0.055035  1.02 -0.54    143
Autumn Sep    0.058322  0.99 -0.58    143
       Oct    0.084755  1.09 -0.57    143
       Nov    0.077762  1.11 -0.55    143
Winter Jan    0.064236  1.17 -0.81    144
       Feb    0.071806  1.37 -0.63    144
       Dec    0.052657  1.16 -0.82    143

In [49]:
grouped_by_season_month = df_global_by_month\
    .groupby(['season','month'],observed=True)\
    .agg(
        avg_anomaly=('anomaly', 'mean'),
        max_anomaly=('anomaly', 'max'),
        min_anomaly=('anomaly', 'min'),
        count_anomaly= ('anomaly', 'count'))
grouped_by_season_month = grouped_by_season_month.reset_index()
grouped_by_season_month.head(3)

,season,month,avg_anomaly,max_anomaly,min_anomaly,count_anomaly
0,Spring,Mar,0.090069,1.36,-0.63,144
1,Spring,Apr,0.056783,1.13,-0.59,143
2,Spring,May,0.047343,1.02,-0.55,143


#### sort

In [50]:
grouped_by_season = df_global_by_month\
    .groupby(['season','month'], observed=True)\
    .agg(avg_anomaly=('anomaly', 'mean'),
         max_anomaly=('anomaly', 'max'),
         count_anomaly=('anomaly', 'count'))\
    .round(3)\
    .sort_values(by='avg_anomaly',ascending=False)\
    .reset_index()
grouped_by_season.head(3)

,season,month,avg_anomaly,max_anomaly,count_anomaly
0,Spring,Mar,0.090,1.36,144
1,Autumn,Oct,0.085,1.09,143
2,Autumn,Nov,0.078,1.11,143


#### apply function to group

In [51]:
# A custom IQR function
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

In [52]:
winter_iqr = grouped_by_season_month['avg_anomaly'].agg(iqr)

0.018854166666666672

In [53]:
winter_iqr = grouped_by_season_month['avg_anomaly'].agg([iqr,'max','min'])

iqr    0.018854
max    0.090069
min    0.033077
Name: avg_anomaly, dtype: float64

In [54]:
def max_min_diff(x):
    return x.max() - x.min()

In [55]:
max_min =  grouped_by_season_month['avg_anomaly'].agg(max_min_diff)

0.05699252136752136

##### lambda

In [56]:
max_min =  grouped_by_season_month['avg_anomaly'].agg(lambda x: x.max() - x.min())

0.05699252136752136

In [57]:
df_global_by_month.groupby('season')\
    .agg(season_diff=('anomaly', lambda x: x.max() - x.min()),
         season_max=('anomaly','max'),
         season_min=('anomaly','min'))

,season_diff,season_max,season_min
season,,,
Spring,1.99,1.36,-0.63
Summer,1.56,1.02,-0.54
Autumn,1.69,1.11,-0.58
Winter,2.19,1.37,-0.82


In [58]:
df_global_by_month.groupby('season')\
    .agg(avg_largest_5=('anomaly', lambda x: x.nlargest(5).mean()),
         anomaly_max=('anomaly','max'))

,avg_largest_5,anomaly_max
season,,
Spring,1.216,1.36
Summer,0.958,1.02
Autumn,1.058,1.11
Winter,1.222,1.37


In [59]:
grp = df_global_by_month.groupby('month')\
    .agg(pct_gt_1=('anomaly', lambda x: (x>1).mean()*100))\
    .round(2)\
    .sort_values(by='pct_gt_1',ascending=False)   
grp.head(3)

,pct_gt_1
month,
Mar,4.17
Jan,2.78
Apr,2.10


#### iterrows

In [60]:
for idx, row in df_global_grouped_by_month.iterrows():
    print(idx,row['avg_anomaly'])

Jan 0.0642361111111111
Feb 0.07180555555555555
Mar 0.09006944444444444
Apr 0.05678321678321678
May 0.047342657342657336
Jun 0.03307692307692308
Jul 0.05657342657342657
Aug 0.05503496503496503
Sep 0.05832167832167832
Oct 0.08475524475524475
Nov 0.07776223776223776
Dec 0.05265734265734266


### .groupby().filter()
- requires writing a custom function or using lambda
- function must return a single boolean value
- filters passes each group as a df
- each group kept or dropped based on returned boolean value
- end result is rows in original df that meet criteria

In [61]:
# task: returns all rows in group where max anomaly of group > 1.35
# use this for checking results of groupby filter
grp = df_global_by_month.groupby('month')\
    .agg(max_anomaly=('anomaly', 'max'))\
    .reset_index()\
    .sort_values(by='max_anomaly',ascending=False)
grp.head(3)

,month,max_anomaly
1,Feb,1.37
2,Mar,1.36
0,Jan,1.17


#### single step using .groupby().filter()

In [62]:
grp = df_global_by_month.groupby('month')\
    .filter(lambda x: x['anomaly'].max() > 1.35)\
    .round(2)
grp.head(3)

,year,month,season,area,anomaly
date,,,,,
1880-02-01,1880,Feb,Winter,GLB,-0.24
1880-03-01,1880,Mar,Spring,GLB,-0.09
1881-02-01,1881,Feb,Winter,GLB,-0.14


#### comparable steps without group filter

### .groupby().transform()
- similar to .agg()
- returns same number of rows as orginal df as a series
- syntax: df.groupby('grouping col')['transformed col'].transform(func)
- can use string name of pandas function or custom function or lambda

#### add group max to each row

In [63]:
dfgtc = df_global_by_month.copy()
dfgtc['month max'] = dfgtc.groupby('month')['anomaly'].transform('sum')
dfgtc.head(3)

,year,month,season,area,anomaly,month max
date,,,,,,
1880-01-01,1880,Jan,Winter,GLB,-0.18,9.25
1880-02-01,1880,Feb,Winter,GLB,-0.24,10.34
1880-03-01,1880,Mar,Spring,GLB,-0.09,12.97


#### add difference from group mean to each row

In [64]:
dfgtc = df_global_by_month.copy()
dfgtc['month mean'] = dfgtc.groupby('month')['anomaly'].transform('mean').round(2)
dfgtc['diff_mean'] = dfgtc.groupby('month')['anomaly'].transform(lambda x: x - x.mean()).round(2)
dfgtc.head(3)

,year,month,season,area,anomaly,month mean,diff_mean
date,,,,,,,
1880-01-01,1880,Jan,Winter,GLB,-0.18,0.06,-0.24
1880-02-01,1880,Feb,Winter,GLB,-0.24,0.07,-0.31
1880-03-01,1880,Mar,Spring,GLB,-0.09,0.09,-0.18


### pivot table

In [65]:
pt = df_global_by_month.pivot_table(
    index='season',
    columns='month', 
    values='anomaly',
    aggfunc=['mean'],
    fill_value=0,
    margins=True).round(3)
pt

mean                                                                 
month     Jan    Feb   Mar    Apr    May    Jun    Jul    Aug    Sep    Oct   
season                                                                        
Spring  0.000  0.000  0.09  0.057  0.047  0.000  0.000  0.000  0.000  0.000  \
Summer  0.000  0.000  0.00  0.000  0.000  0.033  0.057  0.055  0.000  0.000   
Autumn  0.000  0.000  0.00  0.000  0.000  0.000  0.000  0.000  0.058  0.085   
Winter  0.064  0.072  0.00  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
All     0.064  0.072  0.09  0.057  0.047  0.033  0.057  0.055  0.058  0.085   

                             
month     Nov    Dec    All  
season                       
Spring  0.000  0.000  0.065  
Summer  0.000  0.000  0.048  
Autumn  0.078  0.000  0.074  
Winter  0.000  0.053  0.063  
All     0.078  0.053  0.062

### crosstab
adds ability to normalize (normalize options: index, columns, all)

In [66]:
pd.crosstab(index=df_global_by_month['season'], 
            columns=df_global_by_month['month'], 
            normalize='all').round(3) * 100

month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
season,,,,,,,,,,,,
Spring,0.0,0.0,8.4,8.3,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Summer,0.0,0.0,0.0,0.0,0.0,8.3,8.3,8.3,0.0,0.0,0.0,0.0
Autumn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,8.3,8.3,0.0
Winter,8.4,8.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3


### bins

In [67]:
col_min, col_max = df_global_by_month['year'].agg(['min','max'])
print(col_min, col_max)

1880 2023


In [68]:
dfc = df_global_by_month.copy().reset_index()
bins_20yr = [col_min-1,1880,1900,1920,1940,1960,1980,2000,2020,col_max]
dfc['bins'] = pd.cut(dfc['year'], bins = bins_20yr)
dfc.head(5)

,date,year,month,season,area,anomaly,bins
0,1880-01-01,1880,Jan,Winter,GLB,-0.18,"(1879, 1880]"
1,1880-02-01,1880,Feb,Winter,GLB,-0.24,"(1879, 1880]"
2,1880-03-01,1880,Mar,Spring,GLB,-0.09,"(1879, 1880]"
3,1880-04-01,1880,Apr,Spring,GLB,-0.16,"(1879, 1880]"
4,1880-05-01,1880,May,Spring,GLB,-0.10,"(1879, 1880]"


In [69]:
dfc.groupby('bins')['anomaly'].agg(['mean','count'])

,mean,count
bins,,
"(1879, 1880]",-0.169167,12
"(1880, 1900]",-0.220667,240
"(1900, 1920]",-0.328958,240
"(1920, 1940]",-0.155958,240
"(1940, 1960]",-0.008042,240
"(1960, 1980]",0.016417,240
"(1980, 2000]",0.323375,240
"(2000, 2020]",0.729250,240
"(2020, 2023]",0.888148,27


#### quantile binning .qcut()

In [70]:
dfc = df_global_by_month.copy().reset_index()
dfc['qbins'] =  pd.qcut(dfc['year'], 5)
dfc.groupby('qbins')['anomaly'].agg(['mean','count'])

,mean,count
qbins,,
"(1879.999, 1908.0]",-0.246897,348
"(1908.0, 1937.0]",-0.249397,348
"(1937.0, 1965.0]",-0.007619,336
"(1965.0, 1994.0]",0.149511,348
"(1994.0, 2023.0]",0.679912,339


### rolling

In [71]:
df_global_by_month['anomaly'].rolling(5,min_periods=1).agg(['min', 'max', 'mean','count']).head()
#dfgt['Anomaly'].rolling(5,min_periods=1,center=True).agg(['min', 'max', 'mean','count']).head()

,min,max,mean,count
date,,,,
1880-01-01,-0.18,-0.18,-0.1800,1.0
1880-02-01,-0.24,-0.18,-0.2100,2.0
1880-03-01,-0.24,-0.09,-0.1700,3.0
1880-04-01,-0.24,-0.09,-0.1675,4.0
1880-05-01,-0.24,-0.09,-0.1540,5.0


#### .nunique()

In [72]:
tmp = df_global_by_month[["month", "anomaly"]].groupby("month").nunique()

,anomaly
month,
Jan,96
Feb,91
Mar,88
Apr,87
May,87
Jun,89
Jul,79
Aug,88
Sep,83


<a id='da-resample'></a>
## resample

[Return to Start of Notebook](#da)  

- nnAS year start frequency
- nnYS year start frequency
- nnY year end frequency
- nnA year end frequency
- 10Y = decade

- QS quarter start frequency
- Q quarter end frequency
- MS month start frequency
- M month end frequency
- W weekly frequency
- D calendar day frequency
- H hourly frequency

### .resample().agg()

In [73]:
df_global_by_month.head(1)

,year,month,season,area,anomaly
date,,,,,
1880-01-01,1880,Jan,Winter,GLB,-0.18


In [74]:
type(df_global_by_month.index)

pandas.core.indexes.datetimes.DatetimeIndex

#### year

In [75]:
df_global_resampled_by_year = df_global_by_month[['anomaly']]\
            .resample('YS')\
            .agg(avg_anomaly=('anomaly', 'mean'),
                 max_anomaly=('anomaly', 'max'),
                 min_anomaly=('anomaly', 'min'),
                 count_anomaly= ('anomaly', 'count'))\
            .round(2)\
            .reset_index()\
            .set_index('date')
df_global_resampled_by_year.head(3)

,avg_anomaly,max_anomaly,min_anomaly,count_anomaly
date,,,,
1880-01-01,-0.17,-0.09,-0.24,12
1881-01-01,-0.09,0.06,-0.22,12
1882-01-01,-0.11,0.16,-0.36,12


#### decade

In [76]:
df_global_resampled_by_decade = df_global_by_month[['anomaly']]\
            .resample('10YS')\
            .agg(avg_anomaly=('anomaly', 'mean'),
                 max_anomaly=('anomaly', 'max'),
                 min_anomaly=('anomaly', 'min'),
                 count_anomaly= ('anomaly', 'count'))\
            .round(2)\
            .reset_index()\
            .set_index('date')
df_global_resampled_by_decade.tail(3)

,avg_anomaly,max_anomaly,min_anomaly,count_anomaly
date,,,,
2000-01-01,0.59,1.02,0.25,120
2010-01-01,0.81,1.37,0.45,120
2020-01-01,0.93,1.24,0.64,39


### filter for complete decades

In [77]:
filt = df_global_resampled_by_decade["count_anomaly"]==120
df_global_resampled_by_decade_cd = df_global_resampled_by_decade.loc[filt]
df_global_resampled_by_decade_cd.tail(3)

,avg_anomaly,max_anomaly,min_anomaly,count_anomaly
date,,,,
1990-01-01,0.39,0.88,-0.01,120
2000-01-01,0.59,1.02,0.25,120
2010-01-01,0.81,1.37,0.45,120


#### add decade column

<a id='da-export'></a>
## export

[Return to Start of Notebook](#da)  

In [78]:
save_path = os.path.join(aggregated_data_dir, 'global_by_decade.csv')
df_global_resampled_by_decade.to_csv(save_path, header=True, index=True, sep=',')

In [79]:
save_path = os.path.join(aggregated_data_dir, 'global_by_year.csv')
df_global_resampled_by_year.to_csv(save_path, header=True, index=True, sep=',')

In [80]:
save_path = os.path.join(aggregated_data_dir, 'global_by_season.csv')
df_global_grouped_by_season.to_csv(save_path, header=True, index=True, sep=',')

In [81]:
save_path = os.path.join(aggregated_data_dir, 'global_by_month.csv')
df_global_grouped_by_month.to_csv(save_path, header=True, index=True, sep=',')